#### Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from data_utils import *

#### Load Data

In [ ]:
train_valid_ligids  = np.load('../data/PHARM_TRAIN_X.npy')
train_valid_smiles = np.load('../data/PHARM_TRAIN_SMILES.npy')
train_valid_scores = np.load('../data/Y_train.npy')

print('train_valid_ligids shape: {}'.format(train_valid_ligids.shape))
print('train_valid_smiles shape: {}'.format(train_valid_smiles.shape))
print('train_valid_scores shape: {}'.format(train_valid_scores.shape))

#### Train Validation Test Split

In [ ]:
train_data, validation_data = train_validation_split(train_valid_ligids,
                                                     train_valid_smiles,
                                                     train_valid_scores,
                                                     num_val_lig=3046, 
                                                     num_val_smi=10581)
del train_valid_ligids, train_valid_smiles, train_valid_scores

#### Define Model

In [ ]:
# Hyper Paramters
LEARNING_RATE = .0005
LAMBDA = .001
DROPOUT = .5

# Model
L1_UNITS = 10
L2_UNITS = 10
L3_UNITS = 10
NUM_OUTPUTS = 1

tf.reset_default_graph()

relu = tf.nn.relu
elu = tf.nn.elu
xavier_init = tf.contrib.layers.xavier_initializer()
zero_init = tf.zeros_initializer()
l2_reg = tf.contrib.layers.l2_regularizer(scale=LAMBDA)

with tf.name_scope('inputs') as scope:
    X = tf.placeholder(shape=(None, 176), dtype=tf.float32, name='ligids_smiles')
    Y = tf.placeholder(shape=(None), dtype=tf.float32, name='score')
    training = tf.placeholder_with_default(input=False, shape=(), name='training')
    
with tf.name_scope('hidden_layers') as scope:
    layer1 = tf.layers.dense(inputs=X,
                             units=L1_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer1')
    layer2 = tf.layers.dense(inputs=layer1, 
                             units=L2_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer2')
    layer3 = tf.layers.dense(inputs=layer2, 
                             units=L3_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer3')
    
with tf.name_scope('predicted_score') as scope:
    pred_score = tf.layers.dense(inputs=layer3,
                                 units=NUM_OUTPUTS)
    with tf.get_default_graph().gradient_override_map({"Floor": "Identity"}):
        pred_score = tf.floor(pred_score, name='pred_score')
        
with tf.name_scope('train') as scope:
    mae = tf.losses.absolute_difference(Y,pred_score)
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_loss = tf.reduce_sum(reg_loss)
    mse_loss = tf.losses.mean_squared_error(labels=Y,predictions=pred_score)
    loss = mse_loss+reg_loss
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss)

#### Run Model

In [ ]:
# Misc. constants
NUM_EPOCHS = 5
VALIDATIONS_PER_EPOCH = 5
TRAINING_BATCH_SIZE = 1000
USE_PERCENT_DATA = .01
num_training_batches = int(train_data.num_scores/TRAINING_BATCH_SIZE*USE_PERCENT_DATA)
VALIDATION_BATCH_SIZE = 10000
num_validation_batches = int(validation_data.num_scores/VALIDATION_BATCH_SIZE)
TB_PERIOD = 100

# Saver - Prediction Tensors
tf.get_collection_ref('pred_ops').clear()
tf.get_collection('pred_ops')
tf.add_to_collection('pred_ops', X)
tf.add_to_collection('pred_ops', Y)
tf.add_to_collection('pred_ops', pred_score)
# Saver - Training Tensors
tf.get_collection_ref('train_ops').clear()
tf.get_collection('train_ops')
tf.add_to_collection('train_ops', X)
tf.add_to_collection('train_ops', Y)
tf.add_to_collection('train_ops', loss)
tf.add_to_collection('train_ops', train_op)
saver = tf.train.Saver(max_to_keep=1000)

# Tensorboard - Graph
time_now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
tensorboard_logdir = '../tf_log/run-{}'.format(time_now)
print('tensorboard log_dir: {}'.format(tensorboard_logdir))
writer = tf.summary.FileWriter(tensorboard_logdir)
writer.add_graph(tf.get_default_graph())
# Tensorboard - Summaries
summaries = [tf.summary.scalar('mean_abs_error', mae), 
             tf.summary.scalar('mean_square_error', mse_loss), 
             tf.summary.scalar('regularization', reg_loss), 
             tf.summary.scalar('total_loss', loss)]

# Start Session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):       
        # Train Model
        train_data.reset()
        for step in range(num_training_batches):
            ligids_batch, smiles_batch, scores_batch = train_data.next_batch(TRAINING_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch,smiles_batch), axis=1)
            _, train_loss = sess.run([train_op, loss], feed_dict={X:lig_smi_batch, Y:scores_batch})
            print('{}/{} train_loss: {}'.format(step+1, 
                                                num_training_batches, 
                                                train_loss), end='\r')
            # Tensorboard
            if step%TB_PERIOD == 0:
                s = [sess.run(summary, feed_dict={X:lig_smi_batch, Y:scores_batch}) for summary in summaries]
                [writer.add_summary(summary, step) for summary in s]
                
        # Validation
        validation_loss = []
        mae_value = []
        validation_data.reset()
        for step in range(num_validation_batches): 
            ligids_batch, smiles_batch, scores_batch = validation_data.next_batch(VALIDATION_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch, smiles_batch), axis=1)
            validation_loss_batch, mae_batch = sess.run([loss, mae], 
                                                        feed_dict={X:lig_smi_batch, Y:scores_batch})
            validation_loss.append(validation_loss_batch)
            mae_value.append(mae_batch)
            print('{}/{} validation_loss_batch: {}'.format(step+1, 
                                                           num_validation_batches, 
                                                           validation_loss_batch), end='\r')
        validation_loss = sum(validation_loss)/len(validation_loss)
        mae_value = sum(mae_value)/len(mae_value)
        print('EPOCH: {:<10} | loss: {:<25} | mae: {:<20}'.format(epoch+1, 
                                                                  validation_loss, 
                                                                  mae_value))
        # Save Model w/ name: e{epoch number}_l{loss}
        saver_filename = 'l{}_e{}'.format(validation_loss, epoch)        
        saver.save(sess, '../models/{}/{}'.format(time_now, saver_filename))